In [1]:
## 빅데이터 실습

### 스타벅스매장 입지 분석

#### 데이터 수집

##### 셀레니움 사용 자동화

In [34]:
# 필요 라이브러리 사용등록
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

In [38]:
# 웹 드라이버 스타벅스 웹사이트 오픈
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [37]:
# # 웹드라이버에서 소스 위치의 서울 링크 클릭
# path = 'div.loca_step1_cont > ul > li:nth-child(1) > a'
# driver.find_element(By.CSS_SELECTOR, path).click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=122.0.6261.112)
Stacktrace:
	GetHandleVerifier [0x00007FF60BC5AD32+56930]
	(No symbol) [0x00007FF60BBCF632]
	(No symbol) [0x00007FF60BA8419D]
	(No symbol) [0x00007FF60BACACA2]
	(No symbol) [0x00007FF60BABF798]
	(No symbol) [0x00007FF60BAEBC9A]
	(No symbol) [0x00007FF60BABF09A]
	(No symbol) [0x00007FF60BAEBEB0]
	(No symbol) [0x00007FF60BB081E2]
	(No symbol) [0x00007FF60BAEBA43]
	(No symbol) [0x00007FF60BABD438]
	(No symbol) [0x00007FF60BABE4D1]
	GetHandleVerifier [0x00007FF60BFD6ABD+3709933]
	GetHandleVerifier [0x00007FF60C02FFFD+4075821]
	GetHandleVerifier [0x00007FF60C02818F+4043455]
	GetHandleVerifier [0x00007FF60BCF9766+706710]
	(No symbol) [0x00007FF60BBDB90F]
	(No symbol) [0x00007FF60BBD6AF4]
	(No symbol) [0x00007FF60BBD6C4C]
	(No symbol) [0x00007FF60BBC6904]
	BaseThreadInitThunk [0x00007FFE73E07344+20]
	RtlUserThreadStart [0x00007FFE73F426B1+33]


In [39]:
# 웹드라이버에서 소스 위치 서울링크 클릭
name = 'set_sido_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[0].click()

In [45]:
# # 웹 드라이버에서 소스 위치 부산링크 클릭()
# name = 'set_sido_cd_btn'
# driver.find_elements(By.CLASS_NAME, name)[5].click()

In [41]:
# 웹 드라이버에서 소스 위치 전체링크 클릭
name = 'set_gugun_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[0].click()

#### BeautifulSoup로 html 가져오기

In [42]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [46]:
# 검색결과 전체 받아옴
sbSeoulList = soup.select('li.quickResultLstCon')
len(sbSeoulList)

613

##### 서울 매장 리스트 DataFrame

In [78]:
sbSeoulStores = []
for item in sbSeoulList: # 613번(데이터 수 만큼) 반복
    name = item.select('strong')[0].text.strip()
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    storeType = item.select('i')[0]['class'][0].split('_')[1] # 지점 종류(일반, 리저브, DT, WT)
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1] # 주소 / 하나씩 하면서 정리 해 나갈것
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0] # 전화번호 (대표번호로 변경되어 별 의미없음)

    sbSeoulStores.append([name, lat, lng, storeType, address, tel])
    

In [80]:
len(sbSeoulStores)

613

In [81]:
# DF(DataFrame)로 변경
dfStarBucksSeoul = pd.DataFrame(sbSeoulStores, columns=['매장명','위도','경도','매장타입','주소','전화번호'])

In [82]:
dfStarBucksSeoul

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.5139309,127.0206057,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩,37.494668,127.062583,general,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
...,...,...,...,...,...,...
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232
609,상봉역,37.59689,127.08647,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232
611,양원역,37.6066536267232,127.106359790053,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232


In [86]:
dfStarBucksSeoul.tail()

,매장명,위도,경도,매장타입,주소,전화번호
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232
609,상봉역,37.59689,127.08647,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232
611,양원역,37.6066536267232,127.106359790053,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232
612,중화역,37.60170912407773,127.07841136432036,general,서울특별시 중랑구 봉화산로 35 1층,1522-3232


In [87]:
# 결측치가 없음
dfStarBucksSeoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613 entries, 0 to 612
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     613 non-null    object
 1   위도      613 non-null    object
 2   경도      613 non-null    object
 3   매장타입    613 non-null    object
 4   주소      613 non-null    object
 5   전화번호    613 non-null    object
dtypes: object(6)
memory usage: 28.9+ KB


In [88]:
dfStarBucksSeoul.to_excel('./data/starbucks_seoul_list.xlsx', index=False)

##### 서울 열린 데이터광장 OpenAPI 공공데이터 수집

- 아래 방법 동작X

In [89]:
# 서울열린데이터광장 API 키
SEOUL_API_KEY = '5a66586f68676a77313037615a70464a'

In [90]:
# 서울열린데이터광장 호출 URL
url = f'http://openapi.seoul.go.kr:8088/{SEOUL_API_KEY}/json/SdeTlSccoSigW/1/25'
url

'http://openapi.seoul.go.kr:8088/5a66586f68676a77313037615a70464a/json/SdeTlSccoSigW/1/25'

## 행정구역 데이터(위경도 + 구역)

https://github.com/southkorea/seoul-maps

In [ ]:
## 행정구역 데이터(위경도 + 구역)
